In [1]:

import re
import numpy as np
from gensim.models import KeyedVectors
import pickle
import os 
from datetime import datetime
import random 
from sklearn.metrics.pairwise import cosine_similarity
import nltk
#nltk.download('averaged_perceptron_tagger')
model = KeyedVectors.load_word2vec_format('pretrained_model/wiki-news-300d-1M.vec', binary=False)


/home/zzc/anaconda3/envs/exp/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
# load glove 
from gensim.scripts.glove2word2vec import glove2word2vec
 
# 输入文件
glove_file = 'pretrained_model/glove.6B.300d.txt'
 
# 输出文件
tmp_file = "pretrained_model/glove_word2vec_300d.txt"
 
# 转换成word2vec的词向量
_ = glove2word2vec(glove_file, tmp_file)
 
# 加载转化后的文件
model = KeyedVectors.load_word2vec_format(tmp_file, binary=False)
 
import _pickle 
# 序列化
with open('pretrained_model/pickled_model_300d', 'wb') as f:
    _pickle.dump(model, f)
    


/home/zzc/anaconda3/envs/exp/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  # This is added back by InteractiveShellApp.init_path()


In [3]:
import _pickle 
# 测试模型
f = open('pretrained_model/pickled_model_300d', 'rb')
model = _pickle.load(f)
print(model.similarity('have','has'))

0.761861


In [4]:
def mean_vec(vecs):
    total = np.array([0.0]*300)
    for v in vecs:
        total += v
    return total/len(vecs)

In [31]:
# 为AWA2的类产生vector
awa2_class = []
#读取动物列表
with open('data/awa_class.txt') as f:
    lines=f.readlines()
    for i in lines:
        awa2_class.append(i.strip())

vec_dict = {}
print(awa2_class)

for i in awa2_class:
    vecs = []
    for j in i.split('+'):
        try:
            vecs.append(model[j])
        except:
            print('lost', j, '-', i)
    print(i, len(vecs))
    vec_dict[i] = mean_vec(vecs)
    print(vec_dict[i].shape)
    


['antelope', 'grizzly+bear', 'killer+whale', 'beaver', 'dalmatian', 'persian+cat', 'horse', 'german+shepherd', 'blue+whale', 'siamese+cat', 'skunk', 'mole', 'tiger', 'hippopotamus', 'leopard', 'moose', 'spider+monkey', 'humpback+whale', 'elephant', 'gorilla', 'ox', 'fox', 'sheep', 'seal', 'chimpanzee', 'hamster', 'squirrel', 'rhinoceros', 'rabbit', 'bat', 'giraffe', 'wolf', 'chihuahua', 'rat', 'weasel', 'otter', 'buffalo', 'zebra', 'giant+panda', 'deer', 'bobcat', 'pig', 'lion', 'mouse', 'polar+bear', 'collie', 'walrus', 'raccoon', 'cow', 'dolphin']
antelope 1
(300,)
grizzly+bear 2
(300,)
killer+whale 2
(300,)
beaver 1
(300,)
dalmatian 1
(300,)
persian+cat 2
(300,)
horse 1
(300,)
german+shepherd 2
(300,)
blue+whale 2
(300,)
siamese+cat 2
(300,)
skunk 1
(300,)
mole 1
(300,)
tiger 1
(300,)
hippopotamus 1
(300,)
leopard 1
(300,)
moose 1
(300,)
spider+monkey 2
(300,)
humpback+whale 2
(300,)
elephant 1
(300,)
gorilla 1
(300,)
ox 1
(300,)
fox 1
(300,)
sheep 1
(300,)
seal 1
(300,)
chimpanzee 

In [8]:
cub_class = []
with open('data/cub_class.txt') as f:
    lines=f.readlines()
    for line in lines:
        name = line.strip().lower()
        cub_class.append(name)
print(cub_class)

vec_dict = {}

for i in cub_class:
    vecs = []
    for j in i.split('_'):
        try:
            vecs.append(model[j])
        except:
            print('lost', j, '-', i)
    if len(vecs) == 0:
        vec_dict[i] = np.array([0]*300)
    else:
        vec_dict[i] = mean_vec(vecs)


['black_footed_albatross', 'laysan_albatross', 'sooty_albatross', 'groove_billed_ani', 'crested_auklet', 'least_auklet', 'parakeet_auklet', 'rhinoceros_auklet', 'brewer_blackbird', 'red_winged_blackbird', 'rusty_blackbird', 'yellow_headed_blackbird', 'bobolink', 'indigo_bunting', 'lazuli_bunting', 'painted_bunting', 'cardinal', 'spotted_catbird', 'gray_catbird', 'yellow_breasted_chat', 'eastern_towhee', 'chuck_will_widow', 'brandt_cormorant', 'red_faced_cormorant', 'pelagic_cormorant', 'bronzed_cowbird', 'shiny_cowbird', 'brown_creeper', 'american_crow', 'fish_crow', 'black_billed_cuckoo', 'mangrove_cuckoo', 'yellow_billed_cuckoo', 'gray_crowned_rosy_finch', 'purple_finch', 'northern_flicker', 'acadian_flycatcher', 'great_crested_flycatcher', 'least_flycatcher', 'olive_sided_flycatcher', 'scissor_tailed_flycatcher', 'vermilion_flycatcher', 'yellow_bellied_flycatcher', 'frigatebird', 'northern_fulmar', 'gadwall', 'american_goldfinch', 'european_goldfinch', 'boat_tailed_grackle', 'eared_

In [32]:
# 为SUN数据集的类别产生特征文件
from scipy import io as sio
import numpy as np
dataset = 'SUN_data'
image_embedding = 'res101'
class_embedding = 'att' # original_att

path = '../data/SUN_data/'

matcontent = sio.loadmat(path+'res101.mat')

label = matcontent['labels'].astype(int).squeeze() - 1

matcontent = sio.loadmat(path+'att_splits.mat')
trainval_loc = matcontent['trainval_loc'].squeeze() - 1
train_label = np.unique(label[trainval_loc].astype(int))  # 645 

test_unseen_loc = matcontent['test_unseen_loc'].squeeze() - 1
test_label = np.unique(label[test_unseen_loc].astype(int)) # 72 class

with open(path +'trainvalclasses.txt') as f:
    lines=f.readlines()
    train_class = [i.strip() for i in lines]
with open(path +'testclasses.txt') as f:
    lines=f.readlines()
    test_class = [i.strip() for i in lines]
sun_class = []
train_index = 0
test_index = 0
for i in range(717):
    if i in train_label:
        sun_class.append(train_class[train_index])
        train_index += 1
    else:
        sun_class.append(test_class[test_index])
        test_index += 1
print(sun_class[:100])
vec_dict = {}

for i in sun_class:
    vecs = []
    for j in i.split('_'):
        try:
            vecs.append(model[j])
        except:
            print('lost', j, '-', i)
    if len(vecs) == 0:
        vec_dict[i] = np.array([0]*300)
    else:
        vec_dict[i] = mean_vec(vecs)
        
vec_dict['barndoor'] = mean_vec([model['barn'], model['door']])
vec_dict['videostore'] = mean_vec([model['video'], model['store']])
vec_dict['oilrig'] = mean_vec([model['oil']])
vec_dict['thriftshop'] = mean_vec([model['thrift'], model['shop']])
vec_dict['theater_indoor_procenium'] = mean_vec([model['theater'], model['indoor'], model['proscenium']])

['bakery_shop', 'bar', 'barn', 'airlock', 'boathouse', 'bookstore', 'bridge', 'butchers_shop', 'candy_store', 'carrousel', 'temple_south_asia', 'castle', 'cliff', 'closet', 'coast', 'dam', 'dock', 'fastfood_restaurant', 'fly_bridge', 'fountain', 'home_theater', 'house', 'lift_bridge', 'monastery_outdoor', 'piano_store', 'lighthouse', 'manufactured_home', 'mobile_home', 'nursery', 'palace', 'patio', 'pier', 'watering_hole', 'restaurant', 'restaurant_patio', 'roof', 'rope_bridge', 'sandbar', 'geodesic_dome_indoor', 'sawmill', 'sea_cliff', 'seawall', 'roundabout', 'toyshop', 'trestle_bridge', 'underwater_coral_reef', 'underwater_wreck', 'valley', 'viaduct', 'volcano', 'water_tower', 'vegetable_garden', 'chemical_plant', 'chemistry_lab', 'residential_neighborhood', 'tent_indoor', 'utility_room', 'vineyard', 'swimming_hole', 'darkroom', 'shipping_room', 'shed', 'gangplank', 'forest_broadleaf', 'ferryboat_outdoor', 'campus', 'railroad_track', 'auto_racing_paddock', 'zoo', 'garage_outdoor', '

In [52]:
# 为APY数据集的类别产生特征文件
from scipy import io as sio
import numpy as np
dataset = 'APY_data/'

path = '../data/'+dataset

matcontent = sio.loadmat(path+'res101.mat')

label = matcontent['labels'].astype(int).squeeze() - 1

matcontent = sio.loadmat(path+'att_splits.mat')
trainval_loc = matcontent['trainval_loc'].squeeze() - 1
train_label = np.unique(label[trainval_loc].astype(int))  # 645 

print(train_label)
test_unseen_loc = matcontent['test_unseen_loc'].squeeze() - 1
test_label = np.unique(label[test_unseen_loc].astype(int)) # 72 class
print(test_label)

with open(path +'trainvalclasses.txt') as f:
    lines=f.readlines()
    train_class = [i.strip() for i in lines]
with open(path +'testclasses.txt') as f:
    lines=f.readlines()
    test_class = [i.strip() for i in lines]
apy_class = []
print(apy_class[:100])
train_index = 0
test_index = 0
for i in range(32):
    if i in train_label:
        apy_class.append(train_class[train_index])
        train_index += 1
    else:
        apy_class.append(test_class[test_index])
        test_index += 1

vec_dict = {}

for i in apy_class:
    vecs = []
    for j in i.split('_'):
        try:
            vecs.append(model[j])
        except:
            print('lost', j, '-', i)
    if len(vecs) == 0:
        vec_dict[i] = np.array([0]*300)
    else:
        vec_dict[i] = mean_vec(vecs)
        
vec_dict['tvmonitor'] = mean_vec([model['tv'], model['monitor']])
vec_dict['pottedplant'] = mean_vec([model['potted'], model['plant']])
vec_dict['diningtable '] = mean_vec([model['dining'], model['table']])

[ 0  1  2  3  4  5  6  7  8 10 11 17 21 23 25 26 27 29 30 31]
[ 9 12 13 14 15 16 18 19 20 22 24 28]
[]
lost tvmonitor - tvmonitor
lost pottedplant - pottedplant
lost diningtable - diningtable


In [2]:
import scipy.io as sio
import json
import pickle 

from nltk.corpus import wordnet as wn

def getnode(x):
    return wn.synset_from_pos_and_offset('n', int(x[1:]))


base_graph = json.load(open('/home/zzc/exp/zsl_code/19CVPR_GCN_imagenet_DGP/materials/imagenet-induced-graph.json')) # 由下指向上
wnids = base_graph['wnids']
imagenet_class = []
for wnid in wnids:
    name = getnode(wnid).lemma_names()[0]
    imagenet_class.append(name)


In [5]:
vec_dict = {}

for i in imagenet_class:
    vecs = []
    ii = i.lower().replace('_', ' ').replace('-', ' ')
    for j in ii.split(' '):
        try:
            vecs.append(model[j])
        except:
            print('lost', j, '-', i)
    if len(vecs) == 0:
        vec_dict[i] = np.array([0]*300)
    else:
        vec_dict[i] = mean_vec(vecs)

lost dungeness - Dungeness_crab
lost pekinese - Pekinese
lost blenheim - Blenheim_spaniel
lost rhodesian - Rhodesian_ridgeback
lost bluetick - bluetick
lost redbone - redbone
lost ibizan - Ibizan_hound
lost elkhound - Norwegian_elkhound
lost otterhound - otterhound
lost weimaraner - Weimaraner
lost bullterrier - Staffordshire_bullterrier
lost bedlington - Bedlington_terrier
lost sealyham - Sealyham_terrier
lost airedale - Airedale
lost dandie - Dandie_Dinmont
lost dinmont - Dandie_Dinmont
lost clumber - clumber
lost schipperke - schipperke
lost groenendael - groenendael
lost malinois - malinois
lost briard - briard
lost komondor - komondor
lost bouvier - Bouvier_des_Flandres
lost flandres - Bouvier_des_Flandres
lost bernese - Bernese_mountain_dog
lost appenzeller - Appenzeller
lost entlebucher - EntleBucher
lost leonberg - Leonberg
lost samoyed - Samoyed
lost keeshond - keeshond
lost brabancon - Brabancon_griffon
lost pembroke - Pembroke
lost barracouta - barracouta
lost bottlecap - bo

lost pholiota - Pholiota_squarrosa
lost fisherman's - fisherman's_bend
lost astreus - Astreus_hygrometricus
lost hygrometricus - Astreus_hygrometricus
lost garand - Garand_rifle
lost greyhen - greyhen
lost lepiota - lepiota
lost turk's - Turk's_head
lost clitocybe - Clitocybe_subconnexa
lost subconnexa - Clitocybe_subconnexa
lost grevy's - grevy's_zebra
lost pelvimeter - pelvimeter
lost viand - viand
lost marrowfat - marrowfat_pea
lost globigerina - globigerina
lost arthrospore - arthrospore
lost guimpe - guimpe
lost ixodes - Ixodes_spinipalpis
lost spinipalpis - Ixodes_spinipalpis
lost litterbin - litterbin
lost eames - Eames_chair
lost schizopetalon - schizopetalon
lost mobcap - mobcap
lost kalansuwa - kalansuwa
lost jaconet - jaconet
lost testacean - testacean
lost sensitometer - sensitometer
lost viperine - viperine_grass_snake
lost plumcot - plumcot
lost tamarau - tamarau
lost teju - teju
lost archaeornis - archaeornis
lost stayman - Stayman
lost thriftshop - thriftshop
lost grega

lost sphenisciform - sphenisciform_seabird
lost fasciolopsis - Fasciolopsis_buski
lost beaugregory - beaugregory
lost spirillum - spirillum
lost missel - missel_thrush
lost gueridon - gueridon
lost birdcall - birdcall
lost earthball - earthball
lost emilion - Saint_Emilion
lost rissole - rissole
lost purloo - purloo
lost loofa - loofa
lost fucoid - fucoid
lost calceolaria - calceolaria
lost hygrophorus - Hygrophorus_russula
lost genip - genip
lost rhone - Rhone_wine
lost jacksnipe - jacksnipe
lost quadraphony - quadraphony
lost bitok - bitok
lost nephoscope - nephoscope
lost hoecake - hoecake
lost molter - molter
lost bullshot - bullshot
lost abrocome - abrocome
lost arame - arame
lost phalanger - flying_phalanger
lost craniometer - craniometer
lost newburg - lobster_Newburg
lost moneran - moneran
lost cubitiere - cubitiere
lost carbonnade - carbonnade_flamande
lost krummhorn - krummhorn
lost wallbanger - Harvey_Wallbanger
lost guard's - guard's_van
lost gamebag - gamebag
lost ram's - 

lost hare's - Canary_Island_hare's_foot_fern
lost spodoptera - Spodoptera_exigua
lost marang - marang
lost dyer's - dyer's_rocket
lost tachistoscope - tachistoscope
lost talinum - pigmy_talinum
lost glyptics - glyptics
lost butcher's - butcher's_broom
lost smiledon - Smiledon_californicus
lost runcinate - runcinate_leaf
lost strafer - strafer
lost curlycup - curlycup_gumweed
lost felwort - felwort
lost thysanopter - thysanopter
lost pargeting - pargeting
lost nun's - nun's_habit
lost taracahitian - Taracahitian
lost montia - broad-leaved_montia
lost outdoorswoman - outdoorswoman
lost daisyleaf - daisyleaf_grape_fern
lost dasyure - eastern_dasyure
lost supplejack - supplejack
lost threadfish - threadfish
lost touraco - touraco
lost blackheart - blackheart
lost potboy - potboy
lost arctiid - arctiid
lost cangue - cangue
lost gaillardia - gaillardia
lost connecticuter - Connecticuter
lost hart's - hart's-tongue
lost bristlegrass - bristlegrass
lost chigetai - chigetai
lost tachymeter - ta

lost masterwort - greater_masterwort
lost silverweed - silverweed
lost haulm - haulm
lost burgrass - burgrass
lost calamint - large-flowered_calamint
lost pinesap - pinesap
lost fraxinella - fraxinella
lost stooper - stooper
lost bird's - climbing_bird's_nest_fern
lost architeuthis - architeuthis
lost princewood - princewood
lost souari - souari
lost catostomid - catostomid
lost sloganeer - sloganeer
lost driftfish - driftfish
lost regnellidium - regnellidium
lost yautia - yautia
lost cowpen - cowpen_daisy
lost knawel - knawel
lost cabinetwork - cabinetwork
lost lychnis - scarlet_lychnis
lost shellflower - shellflower
lost balker - balker
lost albatrellus - Albatrellus_ovinus
lost ovinus - Albatrellus_ovinus
lost oligoporus - Oligoporus_leucospongia
lost leucospongia - Oligoporus_leucospongia
lost guimpe - guimpe
lost ozark - Ozark_chinkapin
lost ravenna - Ravenna_grass
lost beggar's - beggar's_lice
lost calostoma - Calostoma_lutescens
lost pricket - pricket
lost autoradiograph - autor

...

lost tarwood - tarwood
lost pyrograph - pyrograph
lost lisper - lisper
lost moreton - Moreton_Bay_tulipwood
lost calliopsis - calliopsis
lost nicol - Nicol_prism
lost plumcot - plumcot
lost lecanora - lecanora
lost pleximeter - pleximeter
lost inflater - inflater
lost o' - light-o'-love
lost goat's - goat's_rue
lost beggarwoman - beggarwoman
lost percheron - Percheron
lost nailfile - nailfile
lost domatium - domatium
lost polistes - Polistes_annularis
lost stillroom - stillroom
lost queen's - Queen's_crape_myrtle
lost monocarp - monocarp
lost grouseberry - grouseberry
lost pecopteris - pecopteris
lost marylander - Marylander
lost analogist - analogist
lost allamanda - allamanda
lost ommastrephes - ommastrephes
lost adder's - yellow_adder's_tongue
lost machinist's - machinist's_vise
lost officer's - officer's_mess
lost verpa - Verpa
lost bevatron - bevatron
lost beefwood - beefwood
lost zealander - New_Zealander
lost read/write - read/write_head
lost girandole - girandole
lost onomancer

lost barrenwort - barrenwort
lost singletary - singletary_pea
lost krubi - krubi
lost wormseed - American_wormseed
lost blite - strawberry_blite
lost tendergreen - tendergreen
lost fumitory - fumitory
lost ladies' - ladies'_tobacco
lost cat's - cat's_foot
lost elecampane - elecampane
lost melosa - melosa
lost lion's - lion's_foot
lost pteropogon - pteropogon
lost costusroot - costusroot
lost costmary - costmary
lost loasa - loasa
lost throatwort - throatwort
lost awnless - awnless_bromegrass
lost medusa's - medusa's_head
lost feathertop - feathertop
lost hardinggrass - hardinggrass
lost squirreltail - squirreltail_barley
lost witchgrass - witchgrass
lost sorgo - sorgo
lost mascarene - mascarene_grass
lost fishpole - fishpole_bamboo
lost felwort - marsh_felwort
lost enchanter's - Alpine_enchanter's_nightshade
lost maranta - maranta
lost rattlebox - American_rattlebox
lost rattlebox - Indian_rattlebox
lost bruno's - St.-Bruno's-lily
lost redstem - redstem_storksbill
lost storksbill - red

lost relievo - alto_relievo
lost relievo - mezzo-relievo
lost synthetism - synthetism
lost workboard - workboard
lost drainboard - drainboard
lost linecut - linecut
lost satinet - satinet
lost moreen - moreen
lost stammel - stammel
lost havelock - havelock
lost capeline - capeline_bandage
lost haik - haik
lost lambrequin - lambrequin
lost camlet - camlet
lost newmarket - newmarket
lost flatwork - flatwork
lost calash - calash
lost biggin - biggin
lost pinner - pinner
lost tuille - tuille
lost tasses - skirt_of_tasses
lost deadlight - deadlight
lost voider - voider
lost paillasse - paillasse
lost underfelt - underfelt
lost falsie - falsie
lost goffer - goffer
lost drygoods - drygoods
lost chalkpit - chalkpit
lost parget - parget
lost jackstraw - jackstraw
lost flushless - flushless_toilet
lost tessella - tessella
lost futtock - futtock_shroud
lost becket - becket
lost packthread - packthread
lost lastex - Lastex
lost marline - marline
lost rabato - rabato
lost backband - backband
lost k

lost mauritanian - Mauritanian
lost angolese - Angolese
lost carthaginian - Carthaginian
lost tswana - Tswana
lost sotho - Sotho
lost chadian - Chadian
lost madagascan - Madagascan
lost leonean - Sierra_Leonean
lost almoravid - Almoravid
lost zambian - Zambian
lost mozambican - Mozambican
lost cairene - Cairene
lost theban - Theban
lost malawian - Malawian
lost swazi - Swazi
lost nigerien - Nigerien
lost yodeller - yodeller
lost wirer - wirer
lost telepathist - telepathist
lost codetalker - codetalker
lost revenuer - revenuer
lost actor's - actor's_agent
lost negotiatress - negotiatress
lost wordmonger - wordmonger
lost paragrapher - paragrapher
lost odist - odist
lost synonymist - synonymist
lost avower - avower
lost laudator - laudator
lost toller - toller
lost haranguer - haranguer
lost vociferator - vociferator
lost prattler - prattler
lost driveller - driveller
lost mentioner - mentioner
lost drawler - drawler
lost attester - attester
lost ejaculator - ejaculator
lost voicer - voi

lost nostoc - nostoc
lost xanthomonad - xanthomonad
lost escherichia - escherichia
lost escherichia - Escherichia_coli
lost erwinia - erwinia
lost pleuropneumonialike - pleuropneumonialike_organism
lost actinomyces - actinomyces
lost ratbite - ratbite_fever_bacterium
lost corynebacterium - corynebacterium
lost pseudomonad - pseudomonad
lost nitrosobacteria - nitrosobacteria
lost treponema - treponema
lost leptospira - leptospira
lost bladderwrack - bladderwrack
lost gulfweed - gulfweed
lost conferva - conferva
lost machupo - Machupo_virus
lost junin - Junin_virus
lost choriomeningitis - lymphocytic_choriomeningitis_virus
lost infusorian - infusorian
lost vorticella - vorticella
lost actinopod - actinopod
lost haemosporidian - haemosporidian
lost myxosporidian - myxosporidian
lost haemoproteid - haemoproteid
lost piroplasm - piroplasm
lost sarcosporidian - sarcosporidian
lost haplosporidian - haplosporidian
lost actinomyxidian - actinomyxidian
lost peridinian - peridinian
lost trichomon

In [6]:
# 有w2v的API产生最相似的词
import nltk 
# 需要指定此处的class_list
class_list = imagenet_class

all_attr = []
lost = []
top_k = 100
dict_name = {}

adj = ['JJ','JJR','JJS']
    
#把所有生成属性存成一维数组
for i in class_list:
    ii = i.lower().replace('_', ' ').replace('-', ' ')
    for j in ii.split(' '):
        try:
            class_attr = model.most_similar(j, topn=top_k)
            class_attr = [ j[0] for j in class_attr]  #只取name 舍弃相似度
            all_attr.extend(class_attr)
        except:
            lost.append(j)
        
#把得到的所有属性中不属于adj的筛选掉
result_adj = [nltk.pos_tag([word])[0][0] for word in all_attr if nltk.pos_tag([word])[0][1] in adj]
print(result_adj[:50])
print('word num(after filtering non-adj word)',len(result_adj))

classes = [j for j in class_list if j not in lost]
print('total class num:',len(class_list))
print(len(lost),'class are incompleted')
print(lost[:50])

['trophy-sized', 'freshly-caught', 'queenfish', 'lumpfish', 'mackeral', 'lancetfish', 'guppy', 'puppy', 'three-eyed', 'ray-finned', 'kibble', 'tremendous', 'fantastic', 'incredible', 'fabulous', 'marvelous', 'good', 'huge', 'greatest', 'enormous', 'terrible', 'big', 'remarkable', 'marvellous', 'considerable', 'nice', 'glorious', 'impressive', 'GREAT', 'extraordinary', 'stupendous', 'outstanding', 'unbelievable', 'significant', 'unparalleled', 'gigantic', 'massive', 'prodigious', 'important', 'substantial', 'biggest', 'horrible', 'large', 'exceptional', 'undoubted', 'valuable', 'horrendous', 'formidable', 'admirable', 'wondrous']
word num(after filtering non-adj word) 514788
total class num: 32324
4112 class are incompleted
['dungeness', 'pekinese', 'blenheim', 'rhodesian', 'bluetick', 'redbone', 'ibizan', 'elkhound', 'otterhound', 'weimaraner', 'bullterrier', 'bedlington', 'sealyham', 'airedale', 'dandie', 'dinmont', 'clumber', 'schipperke', 'groenendael', 'malinois', 'briard', 'komond

In [14]:
#按词频排序
from tqdm import tqdm
from collections import Counter

num_wanted = 1000 #想要的属性数量, awa2-50, cub-312, sun-102, apy-64

counter = Counter(result_adj)
attr_selected = [attr[0] for attr in counter.most_common(num_wanted)]

# 通过cosine相似度为类与生成的属性进行赋值
embedding = np.array([vec_dict[i] for i in class_list])
attr_embedding = []
for attr in attr_selected:
    attr_vec = model[attr]
    attr_embedding.append(attr_vec)
    
attribute_matrix = np.zeros((len(embedding), len(attr_selected)))
for i in range(len(class_list)): 
    for j in range(len(attr_selected)):
        sim = cosine_similarity([embedding[i],attr_embedding[j]])[0][1]
        attribute_matrix[i][j] = sim

attribute_matrix = np.array(attribute_matrix)

In [ ]:
#attribute_matrix = np.array(attribute_matrix)
#np.save('/home/zzc/exp/zsl/generate_attributes/generated_attributes/class_attribute_map_cub_'+str(num_wanted)+'.npy',attribute_matrix)
np.save('generated_attributes_glove/class_attribute_map_imagenet_w2v.npy',attribute_matrix)

In [17]:

sio.savemat('generated_attributes_glove/class_attribute_map_imagenet_w2v.mat',{
    'matrix':attribute_matrix,
    'embedding':np.array(embedding),
    'attribute':attr_selected,
})


In [25]:
from bert_serving.client import BertClient
bc = BertClient()
bc.encode(['panda'])  # a[0][0]: [cls] a[0][1]: [word] a[0][2]: [sep] 

import numpy as np
from tqdm import tqdm

dimension = 1024
embedding = []
for wnid, name in tqdm(zip(wnids, class_list)):
    terms = name.lower().replace('_', ' ').replace('-', ' ').split(' ')
    res = bc.encode(terms)
    embedding.append(np.mean(res, axis=0))

attribute_matrix = np.zeros((len(embedding), len(attr_selected)))
attr_embedding = []
for attr in tqdm(attr_selected):
    terms = attr.lower().replace('_', ' ').replace('-', ' ').strip().split(' ')
    res = bc.encode(terms)
    attr_embedding.append(np.mean(res, axis=0))

for i in tqdm(range(len(wnids))): 
    for j in range(len(attr_selected)):
        sim = cosine_similarity([embedding[i],attr_embedding[j]])[0][1]
        attribute_matrix[i][j] = sim


32324it [56:42,  9.50it/s]


ValueError: all elements in the list must be non-empty string, but element 0 is ''

In [31]:
attribute_matrix = np.zeros((len(embedding), len(attr_selected)))
attr_embedding = []
for attr in tqdm(attr_selected):
    terms = attr.lower().replace('_', ' ').replace('-', ' ').strip().split(' ')
    res = bc.encode(terms)
    attr_embedding.append(np.mean(res, axis=0))

for i in tqdm(range(len(wnids))): 
    for j in range(len(attr_selected)):
        sim = cosine_similarity([embedding[i],attr_embedding[j]])[0][1]
        attribute_matrix[i][j] = sim

100%|██████████| 32324/32324 [2:30:15<00:00,  3.59it/s]  


In [32]:
sio.savemat('generated_attributes_glove/class_attribute_map_imagenet_bert.mat',{
    'matrix':attribute_matrix,
    'embedding':np.array(embedding),
    'attribute':attr_selected,
})


In [36]:
np.array(attr_embedding).shape


(1000, 768)